In [51]:
import os
import zipfile
import urllib.request
import requests
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from h3 import h3

import shapefile
# from shapely.geometry import Polygon
# from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

data_dir = os.path.dirname(os.getcwd()) + '/data'
# nyc_path = data_dir + '/yellow_tripdata_2013.csv'
# nyc_path

In [60]:
url = 'https://data.cityofnewyork.us/resource/uacg-pexx.json'
r = requests.get(url, headers={'X-App-Token': '[tAG4HPvjbfHIoztrQBeUNW3sz]'})

In [85]:
from sodapy import Socrata
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

app_token = 'tAG4HPvjbfHIoztrQBeUNW3sz'

# Authenticated client
client = Socrata('data.cityofnewyork.us',
                 app_token,
                 username='lnhung1999@gmail.com',
                 password='2Th3jung13!')

# results = client.get('uacg-pexx', limit=2000)
results = client.get(
    'uacg-pexx',
    where="tpep_pickup_datetime between '2016-07-01T00:00:00' and '2016-07-31T23:59:59'",
    limit=500_000
)

results_df = pd.DataFrame.from_records(results)

KeyboardInterrupt: 

In [ ]:
results_df

In [19]:
# Let's take a quick look at the file to see what it looks like
df_tmp = pd.read_csv(nyc_path, nrows=5)
print(df_tmp.head())

# Convert the locations to float32 and datetime to str
datatypes = {'Trip_Pickup_DateTime': 'str', 
              'Start_Lon': 'float32',
              'Start_Lat': 'float32'}

# These are the columns I would like to keep
colums = list(datatypes.keys())

# The code will read 5 million rows at one go
chunksize = 5_000_000 

# list to hold the batch dataframe
df_list = [] 

for df_chunk in tqdm(pd.read_csv(nyc_path, chunksize=chunksize, usecols=colums)):       
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk)

35it [03:43,  6.38s/it]


In [20]:
# Merge them all
taxi_df = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173179759 entries, 0 to 173179758
Data columns (total 3 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Trip_Pickup_DateTime  object 
 1   Start_Lon             float64
 2   Start_Lat             float64
dtypes: float64(2), object(1)
memory usage: 3.9+ GB


In [37]:
# Download the location Data
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "../data/taxi_zones.zip")
# with zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:
#     zip_ref.extractall("./shape")

('../data/taxi_zones.zip', <http.client.HTTPMessage at 0x7f3cdf58d150>)

In [ ]:
# Download the Trip Record Data
for month in range(1,7):
    urllib.request.urlretrieve('https://s3.amazonaws.com/nyc-tlc/trip+data/'+ \
                               f'yellow_tripdata_2019-{month:0=2d}.csv', 
                               f'{data_dir}/nyc.2019-{month:0=2d}.csv'
                              )